In [1]:
!pip install stanfordcorenlp
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 5.4 MB/s 


### 1. Coref Links Generation

In [1]:
import pickle5 as pickle

In [2]:
with open('coref_links/snippets_q.pickle','rb') as handle:
  dict_snippets = pickle.load(handle)

In [3]:
len(dict_snippets['snippets'])

11

In [4]:
import json
def coreference_links(text,nlp_server,prop):
#    nlp = StanfordCoreNLP('http://localhost:9001')
#    props = {'annotators':'dcoref','outputFormat':'json','ner.useSUTime':'false','timeout': 800000}
    

    inter_links = 0
    intra_links = 0

    out = nlp_server.annotate(text, properties=prop)
    
    try:
      output = json.loads(out)
      
      corefs = []
      for i in output['corefs']:
          if len(output['corefs'][i]) > 1:
              corefs.append(output['corefs'][i])
        
      sent_nums = []
      for i in corefs:
          arr_nums = [j['sentNum'] for j in i]
          sent_nums.append(arr_nums)
    
      for i in sent_nums:
          for j in range(len(i)):
              if j+1 in range(len(i)):
                  if i[j] == i[j+1]:
                      intra_links = intra_links + 1
                  else:
                      inter_links = inter_links + 1
              else:
                  break

    except Exception as err:
      print(err)
    finally:
      pass
    
    return intra_links,inter_links

In [5]:
from stanfordcorenlp import StanfordCoreNLP
import os


nlp = StanfordCoreNLP(r'PATH-TO-CORENLP', quiet=False)
props = {'annotators': 'coref', 'pipelineLanguage': 'en'}

snippets = dict_snippets['snippets']

folder_path = 'coref_links/start/'
start_index = len(os.listdir(folder_path))

for index,text in enumerate(snippets[start_index:]):
  dict_coref = {'intra_links':0,'inter_links':0,'coref_links':0}
  print('Snippet ',start_index+index)
  intra_links,inter_links = coreference_links(text,nlp,props)
  dict_coref['intra_links'] = intra_links
  dict_coref['inter_links'] = inter_links
  dict_coref['coref_links'] = inter_links+intra_links
  with open(folder_path+str(index+start_index)+'.pickle','wb') as handle:
      pickle.dump(dict_coref, handle, protocol=pickle.HIGHEST_PROTOCOL)


#text = 'Barack Obama was born in Hawaii.  He is the president. Obama was elected in 2008.'
#result = json.loads(nlp.annotate(text, properties=props))

#num, mentions = result['corefs'].items()[0]
#for mention in mentions:
#    print(mention)

Snippet  0
Snippet  1
Snippet  2
Snippet  3
Snippet  4
Snippet  5
Snippet  6
Snippet  7
Snippet  8
Snippet  9
Snippet  10


##2. Tree Generation

In [1]:
import pickle5 as pickle
with open('coref_links/snippets_q.pickle','rb') as handle:
  dict_snippets_end = pickle.load(handle)

In [2]:
dict_snippets_end['snippets'][3]

'The invention allows for the production of S. achromogenes before subjecting them to an antibiotic challenge. Location of the resultant gene directly in the chromosomal host dna can be aided by the use of temperature-sensitive plasmids or other curable vectors.'

In [3]:
from stanfordcorenlp import StanfordCoreNLP
import os
import re
from nltk.tokenize import sent_tokenize

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
snippets_reformatted = []
snippets = dict_snippets_end['snippets']
for index,snip in enumerate(snippets):
    snip_new = snip.replace(';','.')
    snip_new = snip_new.replace(',','.')
    snip_new = snip_new.replace('..','.')
    snip_new = re.sub(r'[^A-Za-z.\s]', '', snip_new)
    snippet_group = sent_tokenize(snip_new)
    snippets_reformatted.append(snippet_group)

In [6]:
import json

In [7]:
nlp = StanfordCoreNLP(r'PATH-TO-CORENLP', quiet=False)
props = {'annotators': 'parse', 'pipelineLanguage': 'en'}

folder_path = 'coref_links/snippet_trees/start'
start_index = len(os.listdir(folder_path))
for index,i in enumerate(snippets_reformatted[start_index:]):
  print('Snippet ',start_index+index)
  trees = []
  for j in i:
    print(j)
    out = nlp.annotate(j, properties=props)
    try:
      output = json.loads(out)
      parse_tree = output['sentences'][0]['parse']
      trees.append(parse_tree)
    except Exception as err:
      print(err)
    finally:
      pass
  print('\n')
  print
  with open(folder_path+str(index+start_index)+'.pickle','wb') as handle:
      pickle.dump(trees, handle, protocol=pickle.HIGHEST_PROTOCOL)

Snippet  0
The app keeps track of the degree of users compliance and analyzes the data to determine the optimum lengths of work and break intervals.
It reminds users to take appropriate breaks.


Snippet  1
A nursing cushion with builtin privacy panels that gives breastfeeding mothers some private space.
The cushion is ushaped and has a pillow case as well as soft stuffing material.


Snippet  2
The source and drain that are created from the electrode parts are created when they are perfectly aligned.
The gap between the electrodes can be minimized by using spacer technology to create a space between them.


Snippet  3
The invention allows for the production of S. achromogenes before subjecting them to an antibiotic challenge.
Location of the resultant gene directly in the chromosomal host dna can be aided by the use of temperaturesensitive plasmids or other curable vectors.


Snippet  4
The invention involves coating work items in a vacuum treatment system with an arc vaporization sou